In [1]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import h3
from timeit import default_timer as timer
import folium
from modules.config import PATH_DIR_TRIPS_RAW, PATH_TRIPS, H3_RESOLUTION, H3_RESOLUTION_DOWNSCALING_QUANTILES, PATH_HEXAGON_RESOLUTION_MAP

# Data Preperation
## Collect & Merge
Our dataset consists of multiple csv files. In order to process all of them simultaneously, we merge them together in one dataframe.

In [3]:
start = timer()
dfs = []
for root, subdirs, files in os.walk(PATH_DIR_TRIPS_RAW):
    for file in files:
        path_to_csv = os.path.join(root, file)
        df = pd.read_csv(path_to_csv)
        dfs.append(df)

trips_raw = pd.concat(dfs)

end = timer()
print(f"Succesfully merged csv data into one dataframe in {(end - start):.2f} seconds")

Succesfully merged csv data into one dataframe in 3.08 seconds


In [4]:
trips_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1055091 entries, 0 to 5079
Data columns (total 24 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1055091 non-null  object 
 1   provider             1055091 non-null  object 
 2   city                 1055091 non-null  object 
 3   vehicleType          1055091 non-null  object 
 4   model                616634 non-null   object 
 5   datetime_start       1055091 non-null  object 
 6   date_start           1055091 non-null  int64  
 7   time_start           1055091 non-null  int64  
 8   datetime_end         1055091 non-null  object 
 9   date_end             1055091 non-null  int64  
 10  time_end             1055091 non-null  int64  
 11  longitude_start      1055091 non-null  float64
 12  latitude_start       1055091 non-null  float64
 13  longitude_end        1055091 non-null  float64
 14  latitude_end         1055091 non-null  float64
 15  f

The relevant columns (id, provider, vehicleType, datetime_start, datetime_end, longitude_start, longitude_end, latitude_start, latitude_end, distance) do not have any null values. Therefore we do not have to perform any deletion or imputation strategies.

In [5]:
lons = pd.concat([trips_raw['longitude_start'], trips_raw['longitude_end']])
lats = pd.concat([trips_raw['latitude_start'], trips_raw['latitude_end']])

In [6]:
lat_min = lats.min()
lat_max = lats.max()
lon_min = lons.min()
lon_max = lons.max()

points = [(lat_max, lon_min), (lat_min, lon_min), (lat_min, lon_max), (lat_max, lon_max), (lat_max, lon_min)]

In [7]:
fmap = folium.Map(location=(50.9253, 6.9495), zoom_start=11, control_scale=True, max_zoom=20)
folium.PolyLine(points).add_to(fmap)
fmap

All trips starting and ending locations fall within the blue square. This seems plausible, so there are no outliers in the geospatial data.

In [8]:
# convert string time columns to datetime format
trips_raw['datetime_start'] =  pd.to_datetime(trips_raw['datetime_start'], format='%Y%m%d-%H%M%S')
trips_raw['datetime_end'] =  pd.to_datetime(trips_raw['datetime_end'], format='%Y%m%d-%H%M%S')

In [9]:
trips_raw["start"] = list(zip(trips_raw["latitude_start"],trips_raw["longitude_start"]))
trips_raw["end"] = list(zip(trips_raw["latitude_end"],trips_raw["longitude_end"]))

In [10]:
trips_raw["vehicleType"] = trips_raw["vehicleType"].replace(
    {"kick scooter": "kick_scooter"}
)

To properly examine the geospatial data we will use the library H3, which is a "Hexagonal Hierarchical Spatial Index". This means that the surface of the earth is split into hexagons. We can use these hexagons to easily group spatial data, like our trips starting and end points.  
H3 also provides different resolutions. Choosing a higher resolution results in smaller hexagons, which in our case leads to a more realistic model. However more hexagons also increase the computional power required to create and solve the problem.  
The H3 Resolution is configurable in the `config.py`. For regular hardware we recommand a resultion of 7 (~80 regions) or 6 (~16 regions).

In [11]:
start = timer()
trips_raw["start_hex_id"] = trips_raw.apply(
    lambda row: h3.geo_to_h3(row.latitude_start, row.longitude_start, H3_RESOLUTION),
    axis=1,
)
trips_raw["end_hex_id"] = trips_raw.apply(
    lambda row: h3.geo_to_h3(row.latitude_end, row.longitude_end, H3_RESOLUTION),
    axis=1,
)
end = timer()
print(f"assigned h3 hex ids in {(end - start):.2f} seconds")

assigned h3 hex ids in 30.50 seconds


The number of regions will have a large influence on the dimensionality of the linear program underlying our model. Therefore a large number of regions will result in long runtimes. To reduce the number of regions we have to decrease the h3 resolution. A smaller resolution will however also result in bigger regions, which decreases to accuracy of our model.  
Choosing a single resolution for all of our data leads to some regions at the border of cologne that have very little demand. It is not necessary for these regions to have the same resolution as the inner regions, where most demand is happening.  
Therefore we will downscale (decrease the h3 resolution) for regions that have a low number of incoming and outgoing trips. This downscaling in configurable with the `H3_RESOLUTION_DOWNSCALING_QUANTILES` variable in the `settings.py` file.  
The length of the array determines how often we downscale from the initial `H3_RESOLUTION`. Each entry in the array represents the quantil of regions with the lowest demand, which willbe downscaled.  
E.g. `H3_RESOLUTION = 8` and `H3_RESOLUTION_DOWNSCALING_QUANTILES = [0.9, 0.75]` would mean that we first calculate the hexagons of resolution 8 for all regions and then downscale 90% of the regions with the lowest number of outgoing and incoming trips. We would then repeat the process and downscale 75% of all regions with resolution 7.  
A visualization of the result is available in 1. descriptive analysis.

In [12]:
print(f"Number of regions before downscaling: {trips_raw['start_hex_id'].append(trips_raw['end_hex_id']).nunique()}")

Number of regions before downscaling: 82


In [13]:
trips_raw["start_resolution"] = H3_RESOLUTION
trips_raw["end_resolution"] = H3_RESOLUTION

In [14]:
start = timer()
for i, quantile in enumerate(H3_RESOLUTION_DOWNSCALING_QUANTILES):
    print(f"downscaling resolution {H3_RESOLUTION-i} to {H3_RESOLUTION-i-1} for all regions in {quantile}-quantile")
    trips_by_start_hex = trips_raw[trips_raw["start_resolution"] == H3_RESOLUTION - i]\
        .groupby("start_hex_id").size()
    trips_by_end_hex = trips_raw[trips_raw["start_resolution"] == H3_RESOLUTION - i]\
        .groupby("start_hex_id").size()
    trips_by_hex = trips_by_start_hex.add(trips_by_end_hex, fill_value=0)
    print(f"{len(trips_by_hex)} in current resolution")
    cutoff = trips_by_hex.quantile(quantile)
    to_parent_hex = trips_by_hex[trips_by_hex < cutoff].index
    parent_remap = {child: h3.h3_to_parent(child) for child in to_parent_hex}

    print(f"total number of (start) regions before current downscale: {trips_raw['start_hex_id'].nunique()}")
    trips_raw["start_hex_id"] = trips_raw["start_hex_id"].replace(parent_remap)
    trips_raw["end_hex_id"] = trips_raw["end_hex_id"].replace(parent_remap)
    print(f"total number of (start) regions after current downscale: {trips_raw['start_hex_id'].nunique()}")

    print(f"{trips_raw['start_hex_id'].isin(parent_remap.values()).sum()} trips have downscaled region")
    trips_raw.loc[trips_raw['start_hex_id'].isin(parent_remap.values()), 'start_resolution'] = H3_RESOLUTION - i - 1
    trips_raw.loc[trips_raw['end_hex_id'].isin(parent_remap.values()), 'end_resolution'] = H3_RESOLUTION - i - 1
end = timer()
print(f"downscaled {len(H3_RESOLUTION_DOWNSCALING_QUANTILES)} resolutions in {(end - start):.2f} seconds")

downscaling resolution 7 to 6 for all regions in 0.9-quantile
80 in current resolution
total number of (start) regions before current downscale: 80
total number of (start) regions after current downscale: 27
231964 trips have downscaled region
downscaled 1 resolutions in 7.70 seconds


In [15]:
print(f"Number of remaining regions after downscaling: {trips_raw['start_hex_id'].append(trips_raw['end_hex_id']).nunique()}")

Number of remaining regions after downscaling: 29


We also need to save the resolution of each hexagon that is used, because our model will use the resolution to calculate the distances of round trips.

In [16]:
res_per_start_hex_id = trips_raw[['start_hex_id', 'start_resolution']].groupby('start_hex_id').first()\
    .rename_axis(index="hex_id").rename(columns={'start_resolution': 'resolution'})
res_per_end_hex_id = trips_raw[['end_hex_id', 'end_resolution']].groupby('end_hex_id').first()\
    .rename_axis(index="hex_id").rename(columns={'end_resolution': 'resolution'})
res_per_hex_id = res_per_end_hex_id.append(res_per_start_hex_id).reset_index().drop_duplicates()
res_per_hex_id = res_per_hex_id.set_index('hex_id')

To persist data between notebooks we save dataframes to the pickle format.  
The pickle format has the advantage that it can serialize data and therefore we can easily save python specific datatypes like datetimes.

In [17]:
start = timer()

os.makedirs(os.path.dirname(PATH_HEXAGON_RESOLUTION_MAP), exist_ok=True)
res_per_hex_id.to_pickle(PATH_HEXAGON_RESOLUTION_MAP)

os.makedirs(os.path.dirname(PATH_TRIPS), exist_ok=True)
trips_raw.to_pickle(PATH_TRIPS)
end = timer()
print(f"Succesfully saved dataframe to pickle in {(end - start):.2f} seconds")

Succesfully saved dataframe to pickle in 1.43 seconds
